In [ ]:
# Import required libraries
import pandas_datareader as pdr
import math


import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import yfinance as yf
yf.pdr_override()
from pandas_datareader import data as pdr
# Define start and end dates
end_date = '2018-01-01'
start_date = '2001-01-01'

# Get the close prices for GBPUSD
df_gbpusd = pdr.get_data_yahoo('GBPUSD=X', start=start_date, end=end_date)
data_gbpusd = df_gbpusd.values
yfilter = df_gbpusd.filter(['Close']).values
# Get the close prices for SPX500
df_spx500 = pdr.get_data_yahoo('^GSPC', start=start_date, end=end_date)
data_spx500 = df_spx500.values

# Find the minimum length between the two datasets
min_length = min(len(data_gbpusd), len(data_spx500))

# Truncate the longer dataset to match the minimum length
data_gbpusd = data_gbpusd[:min_length]
data_spx500 = data_spx500[:min_length]

# Take every 7th element from the truncated datasets
data_gbpusd = data_gbpusd[::]
data_spx500 = data_spx500[::]




# Get the close prices for USDMXN (MXN=X)
df_usdmxn = pdr.get_data_yahoo('MXN=X', start=start_date, end=end_date)
data_usdmxn = df_usdmxn.values
data_usdmxn = data_usdmxn[:min_length]
data_usdmxn = data_usdmxn[::]

# Get the close prices for USDCAD (CAD=X)
df_usdcad = pdr.get_data_yahoo('CAD=X', start=start_date, end=end_date)
data_usdcad = df_usdcad.values
data_usdcad = data_usdcad[:min_length]
data_usdcad = data_usdcad[::]

# Get the close prices for USDCNY (CNY=X)
df_usdcny = pdr.get_data_yahoo('CNY=X', start=start_date, end=end_date)
data_usdcny = df_usdcny.values
data_usdcny = data_usdcny[:min_length]
data_usdcny = data_usdcny[::]

# Get the close prices for VIX (%5EVIX)
df_vix = pdr.get_data_yahoo('%5EVIX', start=start_date, end=end_date)
data_vix = df_vix.values
data_vix = data_vix[:min_length]
data_vix = data_vix[::]
def data(dataName):
  df_data = pdr.get_data_yahoo(dataName, start=start_date, end=end_date)
  data_data = df_data.values
  data_data = data_data[:min_length]
  data_data = data_data[::]
  return data_data
# Concatenate all the lists
lists = [data_spx500, data_usdmxn, data_usdcad, data_usdcny, data_vix,
         data('EURUSD%3DX'), data('AUDUSD%3DX'), data('NZDUSD%3DX'),
         data('EURJPY%3DX'), data('GBPJPY%3DX'), data('EURGBP%3DX'), data('EURCAD%3DX'),
         data('EURSEK%3DX'), data('EURCHF%3DX'), data('%5ETNX'), data('%5EDJI'),
         data('%5EIXIC'), data('CL%3DF'), data('GC%3DF'), data('%5EFVX'), data('%5ETYX'),
         data('%5EFTSE'), data('%5ENYA'), data('%5EXAX'), data('%5ERUT'), data('%5EGDAXI'),
         data('%5EFCHI'), data('%5ESTOXX50E'), data('%5EN100'), data('%5EBFX'),
         data('%5EN225'), data('%5EHSI'), data('000001.SS'), data('399001.SZ'), data('%5ESTI'),
         data('%5EAXJO'), data('%5EAORD'), data('%5EBSESN'), data('%5EJKSE'), data('%5EKLSE'),
         data('%5ENZ50'), data('%5EKS11'), data('%5ETWII'), data('%5EGSPTSE'), data('%5EBVSP'),
         data('%5EMXX'), data('%5EIPSA'), data('%5EMERV'), data('%5ETA125.TA'), data('%5EIRX')]


# Find the maximum length among all lists
max_length = max(len(lst) for lst in lists)

# Resize lists to have the maximum length
resized_lists = [np.resize(lst, (max_length, 1)) for lst in lists]

# Concatenate the resized lists
data_combined = np.concatenate(resized_lists, axis=1)
"""
data_combined = np.concatenate((data_gbpusd, data_spx500, data_usdmxn, data_usdcad, data_usdcny, data_vix, data('EURUSD%3DX'), data('AUDUSD%3DX'), data('NZDUSD%3DX'), data('EURJPY%3DX'), data('GBPJPY%3DX'), data('EURGBP%3DX'), data('EURCAD%3DX'), data('EURSEK%3DX'), data('EURCHF%3DX'),  data('%5ETNX'), data('%5EDJI'), data('%5EIXIC'), data('CL%3DF'), data('GC%3DF'), data('%5EFVX'), data('%5ETYX'), data('%5EFTSE'), data('%5ENYA'), data('%5EXAX'), data('%5ERUT'), data('%5EGDAXI'), data('%5EFCHI'), data('%5ESTOXX50E'), data('%5EN100'), data('%5EBFX'), data('%5EN225'), data('%5EHSI'), data('000001.SS'), data('399001.SZ'), data('%5ESTI'), data('%5EAXJO'), data('%5EAORD'), data('%5EBSESN'), data('%5EJKSE'), data('%5EKLSE'), data('%5ENZ50'), data('%5EKS11'), data('%5ETWII'), data('%5EGSPTSE'), data('%5EBVSP'), data('%5EMXX'), data('%5EIPSA'), data('%5EMERV'), data('%5ETA125.TA')), axis=1)
"""
# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_combined)
scalegbpusd = scaler.fit_transform(data_gbpusd)
scaley = scaler.fit_transform(yfilter)
# Define training data length
training_data_len = math.ceil(len(data_combined) * .8)

# Create the training data set
train_data = scaled_data[0:training_data_len, :]
scalegbpusd = scalegbpusd[0:training_data_len, :]
yfilter = yfilter[0:training_data_len, :]
# Split data into x_train and y_train
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i - 60:i, :])
    y_train.append(yfilter[i, 0])

# Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape x_train for LSTM input
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 50))
"""
lastnum = scaler.inverse_transform(model.predict(x_train))[-1]
lastone = (lastnum)**2
lastdata = data_gbpusd[-1]**2
print(lastone - lastdata)
print(lastnum - data_gbpusd[-1])
"""

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

'\nlastnum = scaler.inverse_transform(model.predict(x_train))[-1]\nlastone = (lastnum)**2\nlastdata = data_gbpusd[-1]**2\nprint(lastone - lastdata)\nprint(lastnum - data_gbpusd[-1])\n'

In [ ]:
predictions = model.predict(x_train)
print(len(predictions))

90/90 [==============================] - 16s 163ms/step
2868


In [ ]:
testdata = data_gbpusd[60:training_data_len, :]
testdata = testdata[:, 0]
print(testdata)
print(scaler.inverse_transform(predictions))

[1.85911608 1.86839044 1.89000189 ... 1.52483189 1.50579739 1.5108707 ]


AttributeError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GRU
# Build the model
model = Sequential()
model.add(GRU(500, return_sequences=True, input_shape=(x_train.shape[1], 50)))
model.add(GRU(250, return_sequences=False))
model.add(Dense(200))
model.add(Dense(150))
model.add(Dense(100))
model.add(Dense(75))
model.add(Dense(50))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(25))
model.add(Dense(1))



# Compile the model
print('Done adding')
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.Accuracy()])


Done adding


In [ ]:
import keras
from keras.models import Model
from tensorflow.keras.models import load_model


# Load the model from the SavedModel format
model = keras.models.load_model('crutchless8.h5')

In [ ]:
model.fit(x_train, y_train, batch_size=3, epochs=15)

Epoch 1/15
956/956 [==============================] - 227s 238ms/step - loss: 7.6420e-04 - accuracy: 3.4868e-04
Epoch 2/15
956/956 [==============================] - 227s 238ms/step - loss: 1.5995e-04 - accuracy: 0.0000e+00
Epoch 3/15
956/956 [==============================] - 226s 237ms/step - loss: 1.9607e-04 - accuracy: 0.0000e+00
Epoch 4/15
956/956 [==============================] - 226s 237ms/step - loss: 2.1604e-04 - accuracy: 0.0000e+00
Epoch 5/15
956/956 [==============================] - 227s 238ms/step - loss: 2.9198e-04 - accuracy: 0.0000e+00
Epoch 6/15
956/956 [==============================] - 226s 237ms/step - loss: 2.0151e-04 - accuracy: 0.0000e+00
Epoch 7/15
956/956 [==============================] - 242s 253ms/step - loss: 2.6511e-04 - accuracy: 0.0000e+00
Epoch 8/15
956/956 [==============================] - 239s 250ms/step - loss: 6.7103e-04 - accuracy: 0.0000e+00
Epoch 9/15
956/956 [==============================] - 231s 241ms/step - loss: 1.6960e-04 - accuracy: 0.0

In [ ]:
model.predict(x_train)

71/71 [==============================] - 23s 296ms/step


array([[44.958176],
       [45.16594 ],
       [45.325493],
       ...,
       [45.337013],
       [45.36002 ],
       [44.921272]], dtype=float32)

In [ ]:
model.fit(x_train, y_train, batch_size=3, epochs=5)

Epoch 1/5
956/956 [==============================] - 335s 350ms/step - loss: 8.6107e-04 - accuracy: 0.0000e+00
Epoch 2/5
956/956 [==============================] - 341s 357ms/step - loss: 0.0093 - accuracy: 0.0000e+00
Epoch 3/5
956/956 [==============================] - 336s 351ms/step - loss: 0.0038 - accuracy: 0.0000e+00
Epoch 4/5
956/956 [==============================] - 330s 345ms/step - loss: 0.0014 - accuracy: 0.0000e+00
Epoch 5/5
956/956 [==============================] - 335s 350ms/step - loss: 0.0046 - accuracy: 0.0000e+00


In [ ]:
model.fit(x_train, y_train, batch_size=3, epochs=30)

Epoch 1/30
956/956 [==============================] - 365s 382ms/step - loss: 0.0015 - accuracy: 0.0000e+00
Epoch 2/30
956/956 [==============================] - 351s 367ms/step - loss: 0.0013 - accuracy: 0.0000e+00
Epoch 3/30
956/956 [==============================] - 348s 364ms/step - loss: 0.0014 - accuracy: 0.0000e+00
Epoch 4/30
956/956 [==============================] - 343s 359ms/step - loss: 0.0011 - accuracy: 0.0000e+00
Epoch 5/30
956/956 [==============================] - 341s 356ms/step - loss: 0.0018 - accuracy: 0.0000e+00
Epoch 6/30
956/956 [==============================] - 340s 355ms/step - loss: 9.9328e-04 - accuracy: 0.0000e+00
Epoch 7/30
956/956 [==============================] - 339s 354ms/step - loss: 9.4335e-04 - accuracy: 0.0000e+00
Epoch 8/30
956/956 [==============================] - 349s 365ms/step - loss: 8.6606e-04 - accuracy: 0.0000e+00
Epoch 9/30
956/956 [==============================] - 343s 359ms/step - loss: 0.0023 - accuracy: 0.0000e+00
Epoch 10/30
956/

In [ ]:
model.save('crutchless8.h5')

make one where the y_train is the change in slope
